apro il db


In [48]:
import pandas as pd

df = pd.read_csv('dmml_diabetes_db.csv')

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441456 entries, 0 to 441455
Columns: 330 entries, _STATE to _AIDTST3
dtypes: float64(323), object(7)
memory usage: 1.1+ GB


Prima di tutto, basandomi sul codebook in cui la spiegazione delle variabili avviene in ordine, elimino tutte quelle colonne che riguardano le info personali degli intervistati, non essendo necessarie allo scopo di questo studio.

In [27]:
# il conto delle colonne parte da zero 
# voglio eliminare da: 1 a 13
#                       16,17,18, 19, 21, 22 23 24

# ACCORPARE STESSE DOMANDE FATTE SU LINEE DIVERSE. (25 E 15)
# ad esempio numero di adulti in casa è su due colonne differenti a seconda che l'intervista sia avvenuta su linea fissa o mobile



#df.columns.get_loc('NUMADULT_2') # dalla colonna 26 partono le cose mediche 
print(df.columns[[15, 25]])  # nomi delle colonne che vuoi combinare

# CI SONO DOMANDE SU DIVERSE CONDIZIONI MEDICHE E FREQUENZA DI CONTROLLI -> OK

    # DA COLONNA 52 DOMANDE TIPO SESSO ETC
    # NON MI INTERESSA DA COLONNA 55 (RENT OR OWN A HOME) A 58 (CELL NUM)

    #NELLA COLONNA 65 (HEIGHT) L'ALTEZZA è RIPORTATA CON DUE UNITà DI MISURA DIVERSE, VEDERE CHE FARE
    
    #COLONNA 90 E 93 è CATEGORICA PER SPORT PRATICATO
    
    #COLONNA 128 CATEGORICA SU MALATTIA DELLA PERSONA DI CUI CI PRENDIAMO CURA
    
    #COLONNA 131 CATEGORICA

    #DA COLONNA 176 A 184 FATTE SOLO ALLE DONNE
    
    #DA 191 A 197 SOLO AGLI UOMINI
    
    #204 CATEGORICA PER ORIENTAMENTO SESSUALE
    
    
    #DA COLONNA 222 A 230 DOMANDE RELATIVE AL QUESTIONARIO -> DA ELIMINARE
    
    #233 CATEGORICA

    #238 in poi sono variabili calcolate a partire dalle altre singole o combinate
    
    # Supponiamo di voler trovare l'indice della colonna '_PNEUMO2'
col_name = '_LLCPWT'
try:
    col_index = df.columns.get_loc(col_name)
    print(f"L'indice della colonna '{col_name}' è: {col_index}")
except KeyError:
    print(f"La colonna '{col_name}' non esiste nel DataFrame.")


Index(['NUMADULT', 'HHADULT'], dtype='object')
L'indice della colonna '_LLCPWT' è: 237


In [26]:
df["_LLCPWT"].head()

0    341.384853
1    108.060903
2    255.264797
3    341.384853
4    258.682223
Name: _LLCPWT, dtype: float64

In [49]:
# Su DataFrame "pieno", quando gli indici sono ancora validi 
#la 197 eliminata perchè valori buggati
#colonna 237 eliminata perchè valori derivati da altri che abbiamo gia
drop_indices = list(range(1, 14)) + [16, 17, 18, 19, 21, 22, 23, 24] + list(range(55, 59))+ [197] + list(range(222, 231)) + [235, 237]

# Ottieni i nomi delle colonne da salvare
drop_column_names = [df.columns[i] for i in drop_indices]

# Elimina solo se la colonna esiste
safe_to_drop = [col for col in drop_column_names if col in df.columns]

if safe_to_drop:
    print("Eliminando colonne:")
    for col in safe_to_drop:
        print(f"- {col}")
    df = df.drop(columns=safe_to_drop)
else:
    print("Nessuna colonna da eliminare presente.")


Eliminando colonne:
- FMONTH
- IDATE
- IMONTH
- IDAY
- IYEAR
- DISPCODE
- SEQNO
- _PSU
- CTELENUM
- PVTRESD1
- COLGHOUS
- STATERES
- CELLFON3
- NUMMEN
- NUMWOMEN
- CTELNUM1
- CELLFON2
- PVTRESD2
- CCLGHOUS
- CSTATE
- LANDLINE
- RENTHOM1
- NUMHHOL2
- NUMPHON2
- CPDEMO1
- PCDMDECN
- QSTVER
- QSTLANG
- EXACTOT1
- EXACTOT2
- MSCODE
- _STSTR
- _STRWT
- _RAWRAKE
- _WT2RAKE
- _DUALUSE
- _LLCPWT


In [50]:
# vediamo che succede
#ACCORPAMENTO DI 15 E 25 IN UNA SOLA COLONNA (NUMERO DI ADULTI IN CASA, STESSA DOMANDA MA SU LINEA FISSA E MOBILE)

print(df.shape[1])  # numero totale di colonne
# Trova le posizioni (indici) delle due colonne da unire
numadult_lineaFissa = df.columns.get_loc('NUMADULT')
numadult_lineaMobile = df.columns.get_loc('HHADULT')

# Ottieni i nomi delle colonne da usare
col_fissa = df.columns[numadult_lineaFissa]
col_mobile = df.columns[numadult_lineaMobile]

# Esegui solo se la nuova colonna non esiste ancora
if 'NUMADULT_2' not in df.columns:
    try:
        df['NUMADULT_2'] = df[col_fissa].combine_first(df[col_mobile])
        df = df.drop(columns=[col_fissa, col_mobile])
        print("NUMADULT_2 creata con successo.")
    except KeyError:
        print("Attenzione: una delle colonne non esiste più.")
else:
    print("NUMADULT_2 esiste già.")

# Info post-elaborazione
print(f"Numero colonne attuali: {df.shape[1]}")
print("Verifica che le colonne originali siano state eliminate correttamente:")
print(f"- {col_fissa in df.columns = }")
print(f"- {col_mobile in df.columns = }")




293
NUMADULT_2 creata con successo.
Numero colonne attuali: 292
Verifica che le colonne originali siano state eliminate correttamente:
- col_fissa in df.columns = False
- col_mobile in df.columns = False


fatto questo. ci concentriamo sulla variabile target DIABETE3.

In [51]:
#droppo le righe che hanno diabete3 null o pari a 7 o 9 (non lo so/ nessuna risposta)
before = df.shape[0]
df = df[~df['DIABETE3'].isin([7, 9]) & df['DIABETE3'].notna()]
after = df.shape[0]
print(f"Righe eliminate: {before - after}")
print(f"Righe rimanenti: {after}")


Righe eliminate: 798
Righe rimanenti: 440658


In [52]:
#elimino le colonne con più del 50% di NaN

# Soglia di NaN (50%)
threshold = 0.30

# Calcola la percentuale di NaN per colonna
nan_ratio = df.isna().mean()

# Trova colonne da eliminare
cols_to_drop = nan_ratio[nan_ratio > threshold].index.tolist()

# Stampa e rimuovi
if cols_to_drop:
    print("Colonne eliminate per eccesso di NaN (> 30%):")
    for col in cols_to_drop:
        print(f"- {col}")
    df = df.drop(columns=cols_to_drop)
else:
    print("Nessuna colonna supera il 30% di valori NaN.")
    
    
print(f"Numero colonne attuali: {df.shape[1]}")


Colonne eliminate per eccesso di NaN (> 30%):
- LADULT
- CADULT
- POORHLTH
- BPMEDS
- ASTHNOW
- DIABAGE2
- PREGNANT
- SMOKDAY2
- STOPSMK2
- LASTSMK2
- AVEDRNK2
- DRNK3GE5
- MAXDRNKS
- EXRACT11
- EXEROFT1
- EXERHMM1
- EXRACT21
- EXEROFT2
- EXERHMM2
- LMTJOIN3
- ARTHDIS2
- ARTHSOCL
- JOINPAIN
- FLSHTMY2
- IMFVPLAC
- HIVTSTD3
- WHRTST10
- PDIABTST
- PREDIAB1
- INSULIN
- BLDSUGAR
- FEETCHK2
- DOCTDIAB
- CHKHEMO3
- FEETCHK
- EYEEXAM
- DIABEYE
- DIABEDU
- PAINACT2
- QLMENTL2
- QLSTRES2
- QLHLTH2
- CAREGIV1
- CRGVREL1
- CRGVLNG1
- CRGVHRS1
- CRGVPRB1
- CRGVPERS
- CRGVHOUS
- CRGVMST2
- CRGVEXPT
- VIDFCLT2
- VIREDIF3
- VIPRFVS2
- VINOCRE2
- VIEYEXM2
- VIINSUR2
- VICTRCT4
- VIGLUMA2
- VIMACDG2
- CIMEMLOS
- CDHOUSE
- CDASSIST
- CDHELP
- CDSOCIAL
- CDDISCUS
- WTCHSALT
- LONGWTCH
- DRADVISE
- ASTHMAGE
- ASATTACK
- ASERVIST
- ASDRVIST
- ASRCHKUP
- ASACTLIM
- ASYMPTOM
- ASNOSLEP
- ASTHMED3
- ASINHALR
- HAREHAB1
- STREHAB1
- CVDASPRN
- ASPUNSAF
- RLIVPAIN
- RDUCHART
- RDUCSTRK
- ARTTODAY
- ARTHWGT
- A

In [8]:
#check that the data loaded in is in the correct format
pd.set_option('display.max_columns', 500)
df.head()

,_STATE,GENHLTH,PHYSHLTH,MENTHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,BPHIGH4,BLOODCHO,CHOLCHK,TOLDHI2,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA3,CHCSCNCR,CHCOCNCR,CHCCOPD1,HAVARTH3,ADDEPEV2,CHCKIDNY,DIABETE3,SEX,MARITAL,EDUCA,VETERAN3,EMPLOY1,CHILDREN,INCOME2,INTERNET,WEIGHT2,HEIGHT3,QLACTLM2,USEEQUIP,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,SMOKE100,USENOW3,ALCDAY5,FRUITJU1,FRUIT1,FVBEANS,FVGREEN,FVORANG,VEGETAB1,EXERANY2,STRENGTH,SEATBELT,FLUSHOT6,PNEUVAC3,HIVTST6,PCDMDECN,_CHISPNC,_DUALUSE,_LLCPWT,_RFHLTH,_HCVU651,_RFHYPE5,_CHOLCHK,_RFCHOL,_MICHD,_LTASTH1,_CASTHM1,_ASTHMS1,_DRDXAR1,_PRACE1,_MRACE1,_HISPANC,_RACE,_RACEG21,_RACEGR3,_RACE_G1,_AGEG5YR,_AGE65YR,_AGE80,_AGE_G,HTIN4,HTM4,WTKG3,_BMI5,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG,_SMOKER3,_RFSMOK3,DRNKANY5,DROCDY3_,_RFBING5,_DRNKWEK,_RFDRHV5,FTJUDA1_,FRUTDA1_,BEANDAY_,GRENDAY_,ORNGDAY_,VEGEDA1_,_MISFRTN,_MISVEGN,_FRTRESP,_VEGRESP,_FRUTSUM,_VEGESUM,_FRTLT1,_VEGLT1,_FRT16,_VEG23,_FRUITEX,_VEGETEX,_TOTINDA,MAXVO2_,FC60_,STRFREQ_,PAMISS1_,_PACAT1,_PAINDX1,_PA150R2,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_AIDTST3,NUMADULT_2
0,1.0,5.0,15.0,18.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,4.0,2.0,8.0,88.0,3.0,2.0,280.0,510.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,888.0,305.0,310.0,320.0,310.0,305.0,101.0,2.0,888.0,1.0,1.0,1.0,1.0,b'',NaN,1.0,341.384853,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,9.0,1.0,63.0,5.0,70.0,178.0,12701.0,4018.0,4.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.700000e+01,33.0,6.700000e+01,33.0,17.0,100.0,5.397605e-79,5.397605e-79,1.000000e+00,1.000000e+00,50.0,217.0,2.0,1.0,1.0,1.0,5.397605e-79,5.397605e-79,2.0,2469.0,423.0,5.397605e-79,5.397605e-79,4.0,2.0,3.0,3.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0
1,1.0,3.0,88.0,88.0,2.0,1.0,1.0,4.0,3.0,1.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,6.0,2.0,3.0,88.0,1.0,1.0,165.0,508.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,3.0,888.0,302.0,305.0,302.0,202.0,202.0,304.0,1.0,888.0,3.0,2.0,2.0,2.0,b'',NaN,9.0,108.060903,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,7.0,1.0,52.0,4.0,68.0,173.0,7484.0,2509.0,3.0,2.0,1.0,4.0,1.0,1.0,2.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,7.000000e+00,17.0,7.000000e+00,29.0,29.0,13.0,5.397605e-79,5.397605e-79,1.000000e+00,1.000000e+00,24.0,78.0,2.0,2.0,1.0,1.0,5.397605e-79,5.397605e-79,1.0,2876.0,493.0,5.397605e-79,5.397605e-79,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,4.0,2.0,2.0,2.0,1.0
2,1.0,4.0,15.0,88.0,1.0,2.0,2.0,1.0,3.0,1.0,1.0,1.0,7.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,4.0,2.0,7.0,88.0,99.0,2.0,158.0,511.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',NaN,1.0,255.264797,2.0,9.0,1.0,1.0,2.0,NaN,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,11.0,2.0,71.0,6.0,71.0,180.0,7167.0,2204.0,2.0,1.0,1.0,2.0,9.0,9.0,9.0,9.0,9.000000e+02,9.0,9.990000e+04,9.0,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e+00,4.000000e+00,5.397605e-79,5.397605e-79,NaN,NaN,9.0,9.0,1.0,1.0,1.000000e+00,1.000000e+00,9.0,2173.0,373.0,NaN,9.000000e+00,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN,2.0
3,1.0,5.0,30.0,30.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,3.0,2.0,1.0,4.0,2.0,8.0,1.0,8.0,2.0,180.0,507.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,888.0,555.0,101.0,555.0,301.0,301.0,201.0,2.0,888.0,1.0,1.0,1.0,9.0,b'',NaN,1.0,341.384853,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,9.0,1.0,63.0,5.0,67.0,170.0,8165.0,2819.0,3.0,2.0,2.0,2.0,5.0,4.0,1.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,5.397605e-79,100.0,5.397605e-79,3.0,3.0,14.0,5.397605e-79,5.397605e-79,1.000000e+00,1.000000e+00,100.0,20.0,1.0,2.0,1.0,1.0,5.397605e-79,5.397605e-79,2.0,2469.0,423.0,5.397605e-79,5.397605e-79,4.0,2.0,3.0,3.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,9.0,3.0
4,1.0,5.0,20.0,88.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,3.0,2.0,1.0,5.0,2.0,8.0,88.0,77.

In [58]:
#proviamo a droppare le righe che contengon valori nulli, vediamo quanto resta.

df_clean = df.dropna()
print(f"Righe originali: {df.shape[0]}")
print(f"Righe dopo il drop: {df_clean.shape[0]}")

Righe originali: 440658
Righe dopo il drop: 212463


In [59]:
df_clean.shape


(212463, 131)

Dopo aver visto le colonne con abbastanza dati, verifichiamo le variabili rimaste. 
Tra le variabili rimaste, notiamo come alcune siano variabili derivanti da altre -> le andiamo a eliminare.

Decidiasmo di mantenere solo le originali in modo che meglio le possiamo adattare al nostro caso.

In [57]:
# qui vedo gli indici delle colonne da eliminare
col_name = '_AGE80'
try:
    col_index = df.columns.get_loc(col_name)
    print(f"L'indice della colonna '{col_name}' è: {col_index}")
except KeyError:
    print(f"La colonna '{col_name}' non esiste nel DataFrame.")

L'indice della colonna '_AGE80' è: 75


In [60]:
drop_indices = list(range(56, 69)) + list(range(70, 76)) + list(range(77, 80)) + list(range(82, 86)) + list(range(87, 116)) + list(range(117, 130))

# Ottieni i nomi delle colonne da salvare
drop_column_names = [df_clean.columns[i] for i in drop_indices]

# Elimina solo se la colonna esiste
safe_to_drop = [col for col in drop_column_names if col in df_clean.columns]

if safe_to_drop:
    print("Eliminando colonne:")
    for col in safe_to_drop:
        print(f"- {col}")
    df_clean = df_clean.drop(columns=safe_to_drop)
else:
    print("Nessuna colonna da eliminare presente.")

Eliminando colonne:
- _RFHLTH
- _HCVU651
- _RFHYPE5
- _CHOLCHK
- _RFCHOL
- _MICHD
- _LTASTH1
- _CASTHM1
- _ASTHMS1
- _DRDXAR1
- _PRACE1
- _MRACE1
- _HISPANC
- _RACEG21
- _RACEGR3
- _RACE_G1
- _AGEG5YR
- _AGE65YR
- _AGE80
- HTIN4
- HTM4
- WTKG3
- _RFBMI5
- _CHLDCNT
- _EDUCAG
- _INCOMG
- _RFSMOK3
- DRNKANY5
- DROCDY3_
- _RFBING5
- _DRNKWEK
- _RFDRHV5
- FTJUDA1_
- FRUTDA1_
- BEANDAY_
- GRENDAY_
- ORNGDAY_
- VEGEDA1_
- _MISFRTN
- _MISVEGN
- _FRTRESP
- _VEGRESP
- _FRUTSUM
- _VEGESUM
- _FRTLT1
- _VEGLT1
- _FRT16
- _VEG23
- _FRUITEX
- _VEGETEX
- _TOTINDA
- MAXVO2_
- FC60_
- STRFREQ_
- PAMISS1_
- _PAINDX1
- _PA150R2
- _PA300R2
- _PA30021
- _PASTRNG
- _PAREC1
- _PASTAE1
- _LMTACT1
- _LMTWRK1
- _LMTSCL1
- _RFSEAT2
- _RFSEAT3
- _AIDTST3


In [61]:
df_clean.shape

(212463, 63)

In [62]:
import numpy as np

df_clean.head()
# going to make this ordinal. 0 is for no diabetes or only during pregnancy, 1 is for pre-diabetes or borderline diabetes, 2 is for yes diabetes
mapping_DIABETE3 = {
    1: 'Diabetes',
    2: 'NoDiabetes',
    3: 'NoDiabetes',
    4: "PreDiabetes",
    7: np.nan,
    9: np.nan   
}
df_clean['DIABETE3'] = df_clean['DIABETE3'].map(mapping_DIABETE3)
df_clean.DIABETE3.unique()





array(['NoDiabetes', 'Diabetes', 'PreDiabetes'], dtype=object)

In [63]:
df_clean.DIABETE3.unique()
#df_clean.head()

array(['NoDiabetes', 'Diabetes', 'PreDiabetes'], dtype=object)

In [ ]:
import numpy as np

def convert_height_to_cm(code):
    # NA codes
    if code in (7777, 9999) or pd.isna(code):
        return np.nan

    # ft/inches codes: 200-711
    if 200 <= code <= 711:
        feet   = code // 100
        inches = code % 100
        total_inches = feet * 12 + inches
        return total_inches * 2.54

    # metric codes: 9000-9998
    if 9000 <= code <= 9998:
        return code - 9000

    # tutti gli altri casi li consideriamo mancanti
    return np.nan


def convert_weight_to_kg(code):
    # Valori mancanti / “usa-e-getta”
    if pd.isna(code) or code in (7777, 9999):
        return np.nan

    # Codici in libbre: 50–999 → lb
    if 50 <= code <= 999:
        return code * 0.453592  # lb → kg

    # Codici metrici: 9000–9998 → kg direttamente (code − 9000)
    if 9000 <= code <= 9998:
        return code - 9000

    # Tutti gli altri casi li consideriamo missing
    return np.nan

# Applichiamo al DataFrame
df_clean['WEIGHT_KG'] = df_clean['WEIGHT2'].apply(convert_weight_to_kg)



# Applichiamo la funzione al DataFrame
df_clean['HEIGHT_CM'] = df_clean['HEIGHT3'].apply(convert_height_to_cm)



In [65]:
df_clean.describe(include="all")

,_STATE,GENHLTH,PHYSHLTH,MENTHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,BPHIGH4,BLOODCHO,CHOLCHK,TOLDHI2,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA3,CHCSCNCR,CHCOCNCR,CHCCOPD1,HAVARTH3,ADDEPEV2,CHCKIDNY,DIABETE3,SEX,MARITAL,EDUCA,VETERAN3,EMPLOY1,CHILDREN,INCOME2,INTERNET,WEIGHT2,HEIGHT3,QLACTLM2,USEEQUIP,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,SMOKE100,USENOW3,ALCDAY5,FRUITJU1,FRUIT1,FVBEANS,FVGREEN,FVORANG,VEGETAB1,EXERANY2,STRENGTH,SEATBELT,FLUSHOT6,PNEUVAC3,HIVTST6,_CHISPNC,_RACE,_AGE_G,_BMI5,_BMI5CAT,_SMOKER3,_PACAT1,NUMADULT_2,WEIGHT_KG,HEIGHT_CM
count,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.0,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000,212463.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NoDiabetes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,31.363762,2.541878,60.346564,64.937034,1.058838,1.286615,1.926401,1.422215,2.141808,1.0,1.427218,1.612846,1.941100,1.947817,1.967891,1.878007,1.907174,1.900458,1.937617,1.636996,1.819263,1.975257,NaN,1.573634,2.077708,5.021105,1.862950,3.925705,67.648038,16.173786,1.201461,186.487115,517.871361,1.766063,1.885251,1.963250,1.929211,1.840895,1.962313,1.935335,1.585735,2.966342,513.628726,354.032594,198.059879,300.818142,246.330298,286.956741,212.057972,1.261528,593.759304,1.217017,1.496279,1.950551,1.856592,7.562827,1.821865,4.677836,2829.824699,2.973558,3.367287,2.556911,1.967552,81.608948,169.477547
std,13.489183,1.097481,37.210813,35.842470,0.356481,0.693614,0.340605,1.017817,1.024744,0.0,0.959228,0.676613,0.275731,0.358279,0.299807,0.420956,0.389878,0.377267,0.413750,0.480867,0.500264,0.316486,NaN,0.494549,1.537391,1.017042,0.375313,2.875644,36.604507,27.558859,0.449541,246.875469,36.923635,0.563283,0.354833,0.309358,0.433484,0.480237,0.253889,0.352973,0.613075,0.335665,355.349611,180.063495,112.106825,127.273607,114.069936,121.126808,97.695847,0.486835,367.537070,0.740920,0.575620,1.509623,1.010440,2.868956,2.021965,1.375429,673.179890,0.818610,0.991894,1.621214,2.401366,21.871184,10.424451
min,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,65.000000,300.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,101.000000,1.000000,101.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.

In [67]:
df_clean.columns.tolist()

['_STATE',
 'GENHLTH',
 'PHYSHLTH',
 'MENTHLTH',
 'HLTHPLN1',
 'PERSDOC2',
 'MEDCOST',
 'CHECKUP1',
 'BPHIGH4',
 'BLOODCHO',
 'CHOLCHK',
 'TOLDHI2',
 'CVDINFR4',
 'CVDCRHD4',
 'CVDSTRK3',
 'ASTHMA3',
 'CHCSCNCR',
 'CHCOCNCR',
 'CHCCOPD1',
 'HAVARTH3',
 'ADDEPEV2',
 'CHCKIDNY',
 'DIABETE3',
 'SEX',
 'MARITAL',
 'EDUCA',
 'VETERAN3',
 'EMPLOY1',
 'CHILDREN',
 'INCOME2',
 'INTERNET',
 'WEIGHT2',
 'HEIGHT3',
 'QLACTLM2',
 'USEEQUIP',
 'BLIND',
 'DECIDE',
 'DIFFWALK',
 'DIFFDRES',
 'DIFFALON',
 'SMOKE100',
 'USENOW3',
 'ALCDAY5',
 'FRUITJU1',
 'FRUIT1',
 'FVBEANS',
 'FVGREEN',
 'FVORANG',
 'VEGETAB1',
 'EXERANY2',
 'STRENGTH',
 'SEATBELT',
 'FLUSHOT6',
 'PNEUVAC3',
 'HIVTST6',
 '_CHISPNC',
 '_RACE',
 '_AGE_G',
 '_BMI5',
 '_BMI5CAT',
 '_SMOKER3',
 '_PACAT1',
 'NUMADULT_2',
 'WEIGHT_KG',
 'HEIGHT_CM']

L'indice della colonna 'PCDMDECN' è: 55


In [ ]:
#verificare nelle cat. se c'è uno di questi codici. 

missing_codes = [7, 9, 77, 88, 99, 999, 777, 888] # significano non lo so/refused -> li tratto come nan
#in quelle numeriche no però PHYSHLTH MENTHLTH CHILDREN WEIGHT HEIGHT
#tranne che in _STATE, EMPLOY1, INCOME2 _MRACE1
df_clan.replace(missing_codes, np.nan, inplace=True)


In [ ]:
nominal_col = ['_STATE', ]
ordinal_col = ['GENHLTH']
continuos_col = []

E fin qui ok. Ora devo 
- pulire valori speciali
- 

In [14]:
"""
def pulizia_giorni_codificati(df, colonne):
    
    Pulizia di variabili codificate come:
    1-30 = giorni
    88 = nessun giorno
    77, 99 = don't know / refused
    
    df = df.copy()
    
    for col in colonne:
        if col in df.columns:
            df[col] = df[col].replace({88: 0, 77: None, 99: None})
           
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            print(f"Colonna '{col}' non trovata nel DataFrame.")
    
    return df


def pulizia_binaria_1_2_7_9(df, colonne):
    
    Pulisce le variabili binarie codificate come:
    1 = Yes, 2 = No, 7 = Don’t know, 9 = Refused.
    Trasforma in:
    1 → 1, 2 → 0, 7/9 → NaN
    
    df = df.copy()
    colonne_applicate = []

    for col in colonne:
        if col in df.columns:
            df[col] = df[col].replace({1: 1, 2: 0, 7: np.nan, 9: np.nan})
            
            colonne_applicate.append(col)
        else:
            print(f"⚠️ Colonna '{col}' non trovata nel DataFrame.")

    print(f"✔ Colonne binarie pulite: {colonne_applicate}")
    return df

#questo lavoro da finire dopo, adesso è più importante trasformare i tipi, impostare category dove lo è e tutto




# La colonna state la devo trasformare in 'category', il resto si fa dopo.

#GENHLTH -> ORDINALE (NON STANDARDIZZARE)
df_clean['GENHLTH'] = df_clean['GENHLTH'].replace({7: None, 9: None})
#df_clean = df_clean.dropna(subset=['GENHLTH'])

#PHYSHLTH, MENTHLTH -> numerica, pulizia per interpretazione
colonne_da_pulire = ['PHYSHLTH','MENTHLTH', ]  
df_clean = pulizia_giorni_codificati(df_clean, colonne_da_pulire)
#df_clean = df_clean.dropna(subset=colonne_da_pulire)

# Pulizia delle colonne binarie
colonne_bin = ['EXERANY2', 'SMOKE100', 'DRNKANY5', 'MEDCOST','BLOODCHO', 'TOLDHI2', 'CVDINFR4', 'CVDCRHD4', 'CVDSTRK3', 'ASTHMA3','CHCSCNCR', 'CHCOCNCR', 'CHCCOPD1']  
df_clean = pulizia_binaria_1_2_7_9(df_clean, colonne_bin)
#df_clean = df_clean.dropna(subset=colonne_bin)

#CATEGORICA NOMINALE
#conviene one-hot encoding
df_clean['PERSDOC2'] = df_clean['PERSDOC2'].replace({7: None, 9: None})
#df_clean = df_clean.dropna(subset=['PERSDOC2'])
df_clean = pd.get_dummies(df_clean, columns=['PERSDOC2'], prefix='PERSDOC2', drop_first=True)


#CATEGORICA ORDINALE
df_clean['CHECKUP1'] = df_clean['CHECKUP1'].replace({
    7: None,
    9: None,
    8: 5  # 5+ years
})
df_clean['BPHIGH4'] = df_clean['BPHIGH4'].replace({
    1: 3,  # Yes
    2: 2,  # Yes during pregnancy
    4: 1,  # Borderline
    3: 0,  # No
    7: None,
    9: None
})
df_clean['CHOLCHK'] = df_clean['CHOLCHK'].replace({
    7: None,
    9: None
})


#df_clean = df_clean.dropna(subset=['CHECKUP1', 'BPHIGH4', 'CHOLCHK'])
df_clean['CHOLCHK'] = pd.to_numeric(df_clean['CHOLCHK'], errors='coerce')
df_clean['CHECKUP1'] = pd.to_numeric(df_clean['CHECKUP1'], errors='coerce')
df_clean['BPHIGH4'] = pd.to_numeric(df_clean['BPHIGH4'], errors='coerce')


#RIPARTIRE DA HAVARTH3 INCLUSO 
"""



'\ndef pulizia_giorni_codificati(df, colonne):\n    \n    Pulizia di variabili codificate come:\n    1-30 = giorni\n    88 = nessun giorno\n    77, 99 = don\'t know / refused\n    \n    df = df.copy()\n    \n    for col in colonne:\n        if col in df.columns:\n            df[col] = df[col].replace({88: 0, 77: None, 99: None})\n           \n            df[col] = pd.to_numeric(df[col], errors=\'coerce\')\n        else:\n            print(f"Colonna \'{col}\' non trovata nel DataFrame.")\n    \n    return df\n\n\ndef pulizia_binaria_1_2_7_9(df, colonne):\n    \n    Pulisce le variabili binarie codificate come:\n    1 = Yes, 2 = No, 7 = Don’t know, 9 = Refused.\n    Trasforma in:\n    1 → 1, 2 → 0, 7/9 → NaN\n    \n    df = df.copy()\n    colonne_applicate = []\n\n    for col in colonne:\n        if col in df.columns:\n            df[col] = df[col].replace({1: 1, 2: 0, 7: np.nan, 9: np.nan})\n            \n            colonne_applicate.append(col)\n        else:\n            print(f"⚠️ C